In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import logging
import re
import time
import psana
from scipy.optimize import curve_fit
from argparse import ArgumentParser
import json
import os
import sys
from bokeh.plotting import figure, show, output_file
from bokeh.models import Span, Legend, LegendItem, ColorBar, LinearColorMapper
from bokeh.io import output_notebook
import panel as pn
from scipy.stats import gaussian_kde
output_notebook()

Loading BokehJS ...

In [2]:
def get_r_masks(shape, bins=100):
    """Function to generate radial masks for pixels to include in azav"""
    center = (shape[1] / 2, shape[0] / 2)
    x, y = np.meshgrid(np.arange(shape[1]) - center[0], np.arange(shape[0]) - center[1])
    R = np.sqrt(x**2 + y**2)
    max_R = np.max(R)
    min_R = np.min(R)
    bin_size = (max_R - min_R) / bins
    radii = np.arange(1, max_R, bin_size)
    masks = []
    for i in radii:
        mask = (np.greater(R, i - bin_size) & np.less(R, i + bin_size))
        masks.append(mask)

    return masks
bins=100
exp_run = 'exp=cxilv9518:run=148'
info = re.split('=|:', exp_run)
    #print('inof ', info)
exp = info[1]
hutch = exp[:3]
exp_dir = ''.join(['/reg/d/psdm/', hutch, '/', exp, '/xtc/'])
dsname = ''.join([exp_run, ':smd:', 'dir=', exp_dir])
ds = psana.DataSource(dsname)
detector = psana.Detector('jungfrau4M')
ipm = psana.Detector('CXI-DG2-BMMON-WF')
gdet = psana.Detector('FEEGasDetEnergy')
print(psana.DetNames())
masks = get_r_masks((2203, 2299), bins)
azav_data = np.empty((0, bins), float)
i0_data = []
azav_data=[]
start = time.time()
ped = detector.pedestals(1)[0]
for evt_idx, evt in enumerate(ds.events()):
    raw = detector.raw_data(evt) - ped
    image = detector.image(evt, raw)
    run_azav = np.array([np.mean(image[mask]) for mask in masks])
    azav_data.append(run_azav)
    try:
        i0_data.append(gdet.get(evt).f_12_ENRC())
    except:
        print('missing ', evt_idx)
        i0_data.append(0.)
    if evt_idx == 1000:
        break
print('time to finish ', time.time() - start)

[('CxiEndstation.0:Acqiris.0', 'Acqiris', ''), ('NoDetector.0:Evr.2', 'evr2', ''), ('CxiDsu.0:Opal1000.1', 'opal_spectrometer', ''), ('NoDetector.0:Evr.1', 'evr1', ''), ('CxiDs1.0:Wave8.0', 'CXI-DG2-BMMON-WF', ''), ('EBeam', '', ''), ('FEEGasDetEnergy', '', ''), ('CxiDsu.0:Opal1000.0', 'Timetool', ''), ('CxiDs1.0:Jungfrau.0', 'jungfrau4M', ''), ('ControlData', '', '')]
('missing ', 31)
('missing ', 871)
[array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       4.2120376e-01, 1.8004904e+01, 4.0074169e+01, 4.7179344e+01,
       4.7892048e+01, 4.9005547e+01, 5.1973011e+01, 5.5345390e+01,
       5.8942604e+01, 6.3740520e+01, 7.0571381e+01, 8.0155869e+01,
       9.1128487e+01, 1.0382637e+02, 1.1651474e+02, 1.2913203e+02,
       1.4538455e+02, 1.6718097e+02, 1.9600644e+02, 2.1594260e+02,
       2.2770903e+02, 2.3673711e+02, 2.3377832e+02, 2.2570363e+02,
       2.1311435e+02, 1.9971550e+02, 1.7895621e+02, 1.5553419e+02,
       1.4725673e+02, 1.4237422e+02, 1.3485030e+02, 1.31

In [3]:
EXPRUN = 'exp=cxilv9518:run=148'
INFO = re.split('=|:', EXPRUN)
EXP = INFO[1]
HUTCH = EXP[:3]
# Number of bins
BINS = 100
# Number of events to process
EVENTS = 1000
EXPDIR = ''.join(['/reg/d/psdm/', HUTCH, '/', EXP, '/xtc/'])
DSNAME = ''.join([EXPRUN, ':smd:', 'dir=', EXPDIR])
# Percent of max in histogram to reject for I0 data
LR_THRESH = 0.1
# Number of points at each end of azav to fit
LINE_FIT_POINTS = 50
# Number of bins around peak to include in integrated intensity
DELTA_BIN = 5
logger = logging.getLogger(__name__)

In [43]:
def gaussian(x, a, mean, std, m, b):
    """Equation for gaussian with linear component/offset"""
    return (a * np.exp(-((x - mean) / 2 / std) ** 2)) + (m * x + b)

def fit_line(array, fit_points=LINE_FIT_POINTS):
    """Fit the line from edges of array"""
    azav_len = len(ave_azav)
    x0 = fit_points / 2
    x1 = azav_len - (fit_points / 2)
    y0 = np.mean(ave_azav[:fit_points])
    y1 = np.mean(ave_azav[azav_len - fit_points:])
    m, b = np.polyfit((x0, x1), (y0, y1), 1)

    return m, b

def peak_lr(array_data, threshold=LR_THRESH, bins=BINS):
    """Find max of normal distribution from histogram, search right and left until
    population falls below threshold, will run into problems with bimodal distribution
    """
    hist, edges = np.histogram(array_data, bins=bins)

    # Find peak information
    peak_val = hist.max()
    peak_idx = np.where(hist == peak_val)[0][0]
    peak_bin = edges[peak_idx]

    #search right
    right = np.argmax(hist[peak_idx:] < threshold * peak_val)
    right += peak_idx

    # search left
    left_array = hist[:peak_idx]
    left = peak_idx - np.argmax(left_array[::-1] < threshold * peak_val)

    return hist, edges, peak_idx, left, right

# Filter on azimuthally binned array
def det_azav(azav_data, idxs_use):
    """Get the average azimuthally q binned array from used events"""
    azav_use = azav_data[idxs_use]
    ave_azav = (azav_use.sum(axis=0) / len(azav_use))
    logger.debug('Found azimuthal average arrays for all used events')

    return azav_use, ave_azav

def calc_azav_peak(ave_azav):
    """Get the peak from the gaussian fit with linear offset"""
    azav_len = len(ave_azav)
    # Fit the gaussian with line offset, fails for skewed gaussian
    m, b = fit_line(ave_azav)
    x = np.arange(azav_len)
    mean = np.sum(x * ave_azav) / np.sum(ave_azav)
    std = np.sqrt(np.sum((x - mean) ** 2 / azav_len))

    # Try to get the peak from fit
    try:
        # Guass/w line args
        p0 = [max(ave_azav), mean, std, m, b]
        popt, _ = curve_fit(gaussian, x, ave_azav, p0=p0)
        peak = int(round(popt[1]))
    except Exception as e:
        peak = np.where(ave_azav == np.max(ave_azav))[0][0]

    return peak

def get_integrated_intensity(ave_azav, peak_bin, delta_bin):
    """Get the average integrated intensity"""
    low = peak_bin - delta_bin
    high = peak_bin + delta_bin
    integrated_intensity = ave_azav[low: high].sum(axis=0)

    return integrated_intensity

def peak_in_bins(azav_use, peak, i_dat, delta_bin=DELTA_BIN, low_limit=0.01):
    """Get the peak in the bins we're using"""
    low = peak - delta_bin
    hi = peak + delta_bin
    peak_vals = np.array([azav[low:hi].sum(axis=0) for azav in azav_use])
    peak_idxs = np.where(peak_vals > low_limit)
    peak_vals = peak_vals[peak_idxs]
    ratio = peak_vals.mean() / i_dat.mean()

    return peak_vals, peak_idxs, ratio

def fit_limits(i0_data, peak_vals, i_low, i_high, bins=100):
    """Get the line fit and standard deviation of the plot of i0 vs int_intensities"""
    m, b = np.polyfit(i0_data, peak_vals, 1)
    x = np.linspace(i_low, i_high, bins)
    sigma = np.std(peak_vals)
    y = x * m + b

    return x, y, m, b, sigma

          ###### Bokeh Figures #######

def peak_fig(signal, hist, edges, peak_idx, left, right):
    """General histogram plotter with peak location and left/right limits plotted"""
    low = round(edges[left], 2)
    hi = round(edges[right], 2)
    fig = figure(
        title='Used Intensity Distribution for {0}. Low/Hi: {1}/{2}'.format(signal, low, hi),
        x_axis_label='Intensity Values',
        y_axis_label='Counts'
    )
    fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:])
    left_line = Span(location=edges[left], dimension='height', line_color='black')
    right_line = Span(location=edges[right], dimension='height', line_color='black')
    peak_line = Span(location=edges[peak_idx], dimension='height', line_color='red')
    fig.renderers.extend([left_line, right_line, peak_line])

    return fig

def azav_fig(ave_azav, peak, intensity, delta_bin):
    """Generate the azav fig for html file"""
    x_vals = np.arange(len(ave_azav))
    fig = figure(
        title='Average Azimuthal Binned Array: Center - {0}, min/max - {1}/{2}, intensity - {3}'.format(peak, peak-delta_bin, peak+delta_bin, round(intensity, 2)),
        x_axis_label='Bins',
        y_axis_label='Intensity',
    )

    peak_line = Span(location=peak, dimension='height', line_color='green', line_width=2)
    lower_line = Span(location=peak-delta_bin, dimension='height', line_color='black')
    upper_line = Span(location=peak+delta_bin, dimension='height', line_color='black')
    ave_azav_curve = fig.scatter(x_vals, ave_azav)
    fig.renderers.extend([peak_line, lower_line, upper_line])

    azav_legend = Legend(items=[
        LegendItem(label='Azimuthal Average', renderers=[ave_azav_curve])
    ])
    fig.add_layout(azav_legend)

    return fig

def intensity_hist(intensity_hist, edges):
    fig = figure(
            title='Intensity Histogram'
    )
    fig.quad(top=intensity_hist, bottom=0, left=edges[:-1], right=edges[1:])

    return fig

def intensity_vs_peak_fig(intensity, peak_vals, x, y, slope, intercept, sigma):
    """Simple plot of intensity vs peak value"""
    fig = figure(
        title='Peak value vs Intensity. Slope = {0}, Intercept = {1}'.format(round(slope, 2), round(intercept, 2)),
        x_axis_label='Intensity Monitor Value',
        y_axis_label='Peak Values'
    )
    fig.x_range.range_padding = fig.y_range.range_padding = 0
    h, y_edge, x_edge = np.histogram2d(peak_vals, intensity, bins=100)
    fig.image(image=[h], x=x_edge[0], y=y_edge[0], dh=y_edge[-1]-y_edge[0], dw=x_edge[-1]-x_edge[0], palette="Spectral11")
    color_mapper = LinearColorMapper(palette="Spectral11", low=h.min(), high=h.max())
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
    fig.xaxis.bounds = [i0_low, i0_high]
    fig.add_layout(color_bar, 'right')
    fig.line(x, y, color='red')
    fig.line(x, y - 1 * sigma, color='orange')
    fig.line(x, y + 1 * sigma, color='orange')
    return fig

In [45]:
# Find I0 distribution and filter out unused values
i0_data = np.array(i0_data)
i0_hist, i0_edges, i0_peak_idx, i0_left_idx, i0_right_idx = peak_lr(i0_data)
i0_low = i0_edges[i0_left_idx]
i0_high = i0_edges[i0_right_idx]
i0_idxs = np.where((i0_data > i0_low) & (i0_data < i0_high))
i0_data_use = i0_data[i0_idxs]

# Filter on I0 data to get points we will evaluate
hist_peak, edges_peak, peak_idx, left_peak, right_peak = peak_lr(i0_data_use)
p = peak_fig('gdet', hist_peak, edges_peak, peak_idx, left_peak, right_peak)

# Get the azav value we'll use
azav_use = [azav_data[idx] for idx in i0_idxs[0]]
ave_azav = np.array((np.sum(azav_use, axis=0)) / len(azav_use))
# Find the peak bin from average azav values
peak = calc_azav_peak(ave_azav)
peak=25

# Get the integrated intensity and generate fig
integrated_intensity = get_integrated_intensity(ave_azav, peak, DELTA_BIN)
p1 = azav_fig(ave_azav, peak, integrated_intensity, DELTA_BIN)


# Go back through indices and find peak values for all the intensities
low = peak - DELTA_BIN
high = peak + DELTA_BIN
peak_vals = [azav[low:high].sum(axis=0) for azav in azav_use]
# Now fit I0 vs diffraction intensities
x, y, slope, intercept, sigma = fit_limits(i0_data_use, peak_vals, i0_low, i0_high)
p2 = intensity_vs_peak_fig(i0_data_use, peak_vals, x, y, slope, intercept, sigma)


In [46]:
show(p)
show(p1)
show(p2)

In [56]:
results = {
    'low_slope': slope,
    'low_intercept': intercept,
    'low_i0': i0_low,
    'high_i0': i0_high,
    'ave_i0': i0_data.mean(),
    'ave_peak_intensity': integrated_intensity,
    'peak_bin': peak,
    'delta_bin': DELTA_BIN
}


#After processing
# Peak bin
# low and hi delta_r around peak bin
# Io low bound and high bound
# Average Io
# Average integrated intensity
# slope and intercept of fit of 2D plot
# sigma around line fit
# Run number

print(results)

{'low_intercept': -3630.119742500365, 'ave_peak_intensity': 6332.59, 'delta_bin': 5, 'high_i0': 0.8466769609192851, 'ave_i0': 0.6414089038877722, 'peak_bin': 25, 'low_slope': 15036.018845158875, 'low_i0': 0.41789974458224605}


In [54]:
gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=1000)
gspec[0:3, 0:3] = p
gspec[4:6, 0:3] = p1
gspec[7:9, 0:3] = p2
path = '/cds/home/opr/cxiopr/experiments/cxilv9518/jt_cal/run148/'
if not os.path.isdir(path):
    os.makedirs(path)
gspec.save(''.join([path, 'report.html']))

OSError: [Errno 13] Permission denied: '/cds/home/opr/cxiopr/experiments/cxilv9518/jt_cal'